Tài liệu này mang giấy phép Creative Commons Attribution (CC BY). (c) Nguyễn Ngọc Sáng, Zhukovsky 06/2019.

[@SangVn](https://github.com/SangVn) [@VnCFD](https://vncfdgroup.wordpress.com/)

*Thực hành CFD với Python!*

# Bài 21. Điều kiện biên `supersonic, wall`

Điều kiện biên là một phần không thể thiếu của bất kì chương trình CFD nào và nó có ảnh hưởng lớn tới kết quả tính toán. Có nhiều loại điều kiện khác nhau, ở đây để giải quyết bài toán dòng chảy trên âm đưa ra ở bài 19, ta cần ba điều kiện biên **supersonic_inflow, supersonic_outflow, wall (no_slip)**:

<img src='img\Bai_19_1.png' width=550>


## 1. Điều kiện biên supersonic
Như đã giới thiệu ở phần 2 về đường đặc trưng và cách xác định nghiệm của bài toán phân rã trên biên, trong trường hợp dòng trên âm (chảy từ trái sang phải), các đường đặc trưng đều hướng sang phải, do đó các thông số P trên biên lấy bằng thông số ở bên trái. Ta có:
* Điều kiện biên **supersonic_inflow**: thông số trên biên bằng thông số của dòng tự do P_freestream 
* Điều kiện biên **supersonic_outflow**: thông số trên biên bằng thông số trong ô lưới bên trong side

<br>Sau khi xác định thông số trên biên ta làm tiếp hai bước:
* Tính dòng qua biên
* Căn cứ vào vị trí của ô lưới kề biên là trái hay phải (dựa vào chỉ số của ô lưới 0, 1) để thêm bớt dòng vào tổng dòng trong ô lưới `res`.

In [1]:
# Định nghĩa hàm tính dòng qua mặt trong module functions.py

# Hàm P2F: tính dòng qua mặt (công thức ở bài 18)
def P2F(P, side):
    n = side.normal    #vector pháp tuyến đơn vị của mặt
    vn = n.dot(P[1:3]) #vận tốc vuông góc bề mặt V.n
    F = zeros(4)
    F[0] = P[0] * vn
    F[1] = F[0] * P[1] + P[3] * n[0]
    F[2] = F[0] * P[2] + P[3] * n[1]
    F[3] = F[0] * (P[3] / P[0] * gamma / (gamma - 1.0) + 0.5 * (P[1] ** 2 + P[2] ** 2))
    return F * side.area

Module `boco.py`:

In [4]:
# coding: utf-8

from functions import P2F

# thông số dòng tự do
P_freestream = None

# Hàm thiết lập thông số dòng tự do 
def set_freestream(P):
    global P_freestream
    P_freestream = P
    
# Hàm sign_ic: xác định `thêm hay bớt` dòng ở ô lưới kề bên side
def sign_ic(ic): #ic ~ index_cell; left_cell: ic = 0; right_cell: ic = 1
    if ic == 0: return -1.0 # res -= flux
    else: return 1.0        # res += flux

# Điều kiện dòng chảy vào trên âm
# P_side = P_freestream
def supersonic_inflow(boundary, ic):
    for side in boundary:
        F = P2F(P_freestream, side)
        side.cells[ic].res += sign_ic(ic)*F

# Điều kiện dòng chảy ra trên âm
# P_side = P_in
def supersonic_outflow(boundary, ic):
    for side in boundary:
        F = P2F(side.cells[ic].P, side)
        side.cells[ic].res += sign_ic(ic)*F

## 2. Điều kiện biên wall (no_slip)
Trên biên wall (bề mặt rắn, không thẩm thấu, không nhám, không trượt) vận tốc dòng chảy bằng không, tức là $(u,v)=(0,0)$, đối với áp suất và khối lượng riêng ta lấy gradient $(\partial /\partial n)$ bằng không.

In [2]:
# Điều kiện biên wal_no_slip
# P_side: u=v=0, dp/dn=0, drho/dn=0
def wall_no_slip(boundary, ic):
    for side in boundary:
        P = [side.cells[ic].P[0], 0.0, 0.0, side.cells[ic].P[3]]
        F = P2F(P, side)
        side.cells[ic].res += sign_ic(ic)*F

# ta có `từ điển` điều kiện biên: <tên gọi> : <hàm đkb>
boco_dict = {'supersonic_inflow': supersonic_inflow, 'supersonic_outflow': supersonic_outflow, 'wall': wall_no_slip}

Quay trở lại với bài 20, trong hàm tính dòng qua biên `boco(boundary_ib, ic)` *boco* là một trong ba điều kiện ở trên.

Tiếp theo, chúng ta cần giải bài toán phân rã gián đoạn để tính dòng qua các biên nằm bên trong vùng tính toán.
# [Bài 22. Phương pháp tính dòng qua mặt, xử lý kết quả, module solver](Bai_22.ipynb)